In [ ]:
# #@title Random Code Test
# from sklearn.feature_extraction.text import CountVectorizer
# # list of text documents
# text = "The quick brown fox#  jumped! over@ the lazy. dog?" + "\n" + "i love cookies" + " Anh likes chocolate"
# print(text)
# text = [text]
# print(text[0])
# # create the transform
# # vectorizer = CountVectorizer(token_pattern=r"[\s\S]")
# vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w\w+\b|\S*[^\w\s]\S*|\s+|\r\n")
# # tokenize and build vocab
# vectorizer.fit(text)
# # summarize
# print(vectorizer.vocabulary_)
# # encode document
# vector = vectorizer.transform(text)
# # summarize encoded vector
# print(vector.shape)
# print(type(vector))
# print(vector.toarray())

oned_array = torch.as_tensor([0,1,1])
print(oned_array.shape[0])

3


In [ ]:
pip install gmpy2

     |████████████████████████████████| 3.6 MB 5.0 MB/s 


In [ ]:
pip install essential-generators

     |████████████████████████████████| 9.5 MB 5.4 MB/s 


In [ ]:
pip install bitstring

In [ ]:
#@title Import
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
import torch.optim as optim
import pickle as pkl
from numpy import linalg as LA 
from essential_generators import DocumentGenerator
import hashlib
from bitstring import BitArray

In [ ]:
#@title Constant
INPUT_DIM = 1024
OUT_DIM = 32

In [ ]:
#@title Preprocess
def preprocess(file, isSentence=True):
    if isSentence:
        send_file = [file]
    else:
        send_file = open(file, "r").readlines()
        send_file = ' '.join(send_file)
        send_file = [send_file]
    vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w\w+\b|\S*[^\w\s]\S*|\s+|\r\n")
    # tokenize and build vocab
    vectorizer.fit(send_file)
    # encode document
    vector = vectorizer.transform(send_file)
    # summarize encoded vector
    vector_shape = vector.shape
    doc_feat = torch.tensor(vector.toarray(), dtype=torch.float)
    #Add padding
    if vector_shape[1] < INPUT_DIM:
      pad_size = INPUT_DIM - vector_shape[1]
      padding = (0, pad_size)
      doc_feat = F.pad(doc_feat, padding, "constant", 0)
    elif vector_shape[1] < INPUT_DIM:
      pad_size = vector_shape[1] % INPUT_DIM
      padding = (0, pad_size)
      doc_feat = F.pad(doc_feat, padding, "constant", 0)
      num_chunks = doc_feat.shape[1]/INPUT_DIM
      doc_feat = doc_feat.chunk(num_chunks)
    assert doc_feat.shape[1] == INPUT_DIM, f"document input dim does not match. 1024 expected, got: {doc_feat.shape[1]}"
    return doc_feat





In [ ]:
#@title HashNN

class HashNN(nn.Module):
    def __init__(self, out_size):
        super(HashNN, self).__init__()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512,512)
        self.fc3 = nn.Linear(512, out_size)
        self.sequential = nn.Sequential(self.fc1,
                                        nn.ReLU(),
                                        self.fc2,
                                        nn.ReLU(),
                                        self.fc3)
    def forward(self, x):
        x = self.sequential(x)
        return x


In [ ]:
x_train = np.zeros((10000, INPUT_DIM))
x_target = np.zeros(10000)
gen = DocumentGenerator() 
for i in range(10000):
    send_file = gen.sentence()
    doc_feat = preprocess(send_file)
    x_train[i] = doc_feat
    h = hashlib.new('sha256')
    hash = send_file.encode('utf-8')
    target = hashlib.sha256(hash).digest()[:4]
    bin = int.from_bytes(target, "big")
    x_target[i] = bin
    print(i)


In [ ]:
#@title Model training
hash_model = HashNN(1)
epochs = np.random.randint(100, 500)
optimizer = optim.SGD(hash_model.parameters(), lr=1e-5)
criterion = nn.MSELoss()
for i in range(1000):
  idx = np.random.permutation(len(x_target))
  batch_feat = torch.from_numpy(x_train[idx]).to(dtype=torch.float)
  batch_target = torch.from_numpy(x_target[idx]).to(dtype=torch.float)
  optimizer.zero_grad()
  output = hash_model(batch_feat)
  loss = criterion(output, batch_target)
  loss.backward()
  torch.nn.utils.clip_grad_norm(hash_model.parameters(), max_norm=15)
  optimizer.step()
  print(i)

In [ ]:
#@title Save model
for name, param in hash_model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
torch.save(hash_model.state_dict(), "/content/Model_1.pt")

In [ ]:
#@title Convert to Binary
def convert_to_binary(vector):
  vector_tensor = torch.as_tensor(vector)
  sign = torch.sign(vector_tensor)
  binary_output = torch.relu(sign)  
  return binary_output


In [ ]:
#@title Postprocess Dimensionality Reduction
'''
Use dimensionality reduction
to reduce a bit_len x chunk_size 
matrix to a single bit_len vector 
while retaining as much of the information
from the digest as possible
'''
def reduce_digest(D): 
  # Convert tensy to a numpy vector
  D = D.detach().numpy()
  assert D.shape[0] % OUT_DIM == 0
  # Convert to bit_len x chunk_size numpy array
  temp = np.zeros((OUT_DIM, int(D.shape[0]/OUT_DIM)))
  for i in range(temp.shape[1]):
    for j in range(OUT_DIM):
      temp[j][i] = D[(i * OUT_DIM) + j]
  D = temp
  # Center the data 
  c = np.outer(np.ones(D.shape[1]), D.mean(0))
  D = D - c 
  # Get the top right singular vector
  U, S, Vt = LA.svd(D, full_matrices=True)
  Vt = Vt[:1, :].T  
  # Reconstruct the data
  D = D + c 
  # Project down to 1-dim
  return D @ Vt

In [ ]:
#@title Running Preprocess and Model output
collision = 0
hash_outputs = set()
gen = DocumentGenerator() 
for _ in range(10):
    send_file = gen.sentence()
    doc_feat = preprocess(send_file)
    print(doc_feat)
    hash_output = hash_model(doc_feat).detach().item()
    print(hash_output)
    '0x{0:08X}'.format(int(hash_output))
    # hash_output = torch.squeeze(hash_output.view(1,-1))
    # hash_output = torch.exp(hash_output)
    # if hash_output.shape[0] > OUT_DIM:
    #     hash_output = reduce_digest(hash_output)
    #     hash_output = torch.from_numpy(hash_output)
    print("Hash output: ", hash_output)
    # b = [" ".join(item) for item in binary_out.astype(str) ]
    # b = "".join(b)
    # bytes_hash = int(b, 2).to_bytes((len(b) + 7) // 8, 'big')
    # print(bytes_hash)
    if bytes_hash in hash_outputs:
      print(collision)
    else:
      hash_outputs.add(bytes_hash)
    collision +=1

In [ ]:
!pip install torch==1.3.1+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install inltk

In [ ]:
!pip install torch==1.3.1+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
from inltk.inltk import setup
setup('en') 

In [ ]:
from inltk.inltk import get_similar_sentences

In [ ]:
# sample sentence
sentence1 = "The Quick Brown Fox Jumps Over The Lazy Dog"
sentence2 = "I eat rice"
sentence3 = "She sells seashells"
sentence4 = "We love icecream"
sentence5 = "Greatest Snow on Earth"
sentence6 = "We love the U"
# number of similar sentences to be generated
no_of_variants = 5                               # i want 6*5 similar sentences

# code of language 
code_of_language = "en"                                       # "en" is a code to represent english language

# where degree_of_aug is roughly the percentage of sentence you want to augment, with a default value of 0.1
degree_of_aug = 0.3

result1 = get_similar_sentences(sentence1, no_of_variants, code_of_language, degree_of_aug)
result2 = get_similar_sentences(sentence2, no_of_variants, code_of_language, degree_of_aug)
result3 = get_similar_sentences(sentence3, no_of_variants, code_of_language, degree_of_aug)
result4 = get_similar_sentences(sentence4, no_of_variants, code_of_language, degree_of_aug)
result5 = get_similar_sentences(sentence5, no_of_variants, code_of_language, degree_of_aug)
result6 = get_similar_sentences(sentence6, no_of_variants, code_of_language, degree_of_aug)


In [ ]:
for sentence1 in result1:
  print(sentence1)
for sentence2 in result2:
  print(sentence2)
for sentence3 in result3:
  print(sentence3)
for sentence4 in result4:
  print(sentence4)
for sentence5 in result5:
  print(sentence5)
for sentence6 in result6:
  print(sentence6)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/ltrc_yahoo (2).zip"

with ZipFile(file_name, 'r') as zipping:
  zipping.extractall("/content/Fair-PGRank/yahoo")
  print('Done')

Done


In [ ]:
from sklearn.datasets import load_svmlight_file

x_train, y_train, query_ids_train = load_svmlight_file("/content/Fair-PGRank/yahoo/ltrc_yahoo/set1.train.txt", query_id=True)
x_train = x_train.toarray()
x_train = np.unique(x_train, axis=0)
print(x_train.shape)

(438139, 699)
